In [2]:
import pandas as pd
# import numpy as np

In [3]:
database_type = "/full"

The `patient` table includes general information about the patient admissions (for example, demographics, admission and discharge details). 
See: http://eicu-crd.mit.edu/eicutables/patient/

In [9]:
# loads patient table
patient = pd.read_csv('../eICU' + database_type + '/patient.csv', nrows=999999) 
patient = patient[['patientunitstayid', 'admissionweight']].set_index('patientunitstayid')
patient.head()

,admissionweight
patientunitstayid,
141168,84.3
141178,54.4
141179,NaN
141194,73.9
141196,NaN


The `vitalperiodic` table comprises data that is consistently interfaced from bedside vital signs monitors into eCareManager. 
Data are generally interfaced as 1 minute averages, and archived into the `vitalperiodic` table as 5 minute median values. 
For more detail, see: http://eicu-crd.mit.edu/eicutables/vitalPeriodic/

In [12]:
vitalperiodic = pd.read_csv('../eICU' + database_type + '/vitalPeriodic.csv', nrows=999999)
columns = ['patientunitstayid', 'observationoffset', 'temperature', 'heartrate', 'respiration', 'systemicsystolic']
vitalperiodic = vitalperiodic[columns].set_index('observationoffset')
vitalperiodic = vitalperiodic.sort_values(by='observationoffset')
vitalperiodic.head()

,patientunitstayid,temperature,heartrate,respiration,systemicsystolic
observationoffset,,,,,
-1445,148349,NaN,60.0,20.0,NaN
-1440,148349,NaN,60.0,21.0,NaN
-1435,150049,NaN,96.0,19.0,NaN
-1435,146418,NaN,71.0,23.0,NaN
-1435,148349,NaN,60.0,21.0,NaN


The `vitalAperiodic` table provides invasive vital sign data that is recorded at irregular intervals. 
See: http://eicu-crd.mit.edu/eicutables/vitalAperiodic/

In [14]:
vitalaperiodic = pd.read_csv('../eICU' + database_type + '/vitalAperiodic.csv', nrows=999999)
columns = ['observationoffset', 'patientunitstayid','noninvasivesystolic','cardiacoutput','cardiacinput']
vitalaperiodic = vitalaperiodic[columns].set_index('observationoffset')
vitalaperiodic = vitalaperiodic.sort_values(by='observationoffset')
vitalaperiodic.head()

,patientunitstayid,noninvasivesystolic,cardiacoutput,cardiacinput
observationoffset,,,,
-4320,218289,187.0,NaN,NaN
-4319,218289,178.0,NaN,NaN
-3883,218287,138.0,NaN,NaN
-3659,218287,134.0,NaN,NaN
-3632,218287,147.0,NaN,NaN


The `lab` table provides Laboratory tests that have have been mapped to a standard set of measurements. 
Unmapped measurements are recorded in the customLab table.
See: http://eicu-crd.mit.edu/eicutables/lab/

In [19]:
lab = pd.read_csv('../eICU' + database_type + '/lab.csv', nrows=999999)
columns = ['labresultoffset', 'patientunitstayid','labtypeid','labname', 'labresult', 'labmeasurenamesystem', 'labmeasurenameinterface']
lab = lab[columns].set_index('labresultoffset')
lab = lab.sort_values(by='labresultoffset')
lab.head()

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
0,52307161,141168,2026,3,fibrinogen,177.0,177,mg/dL,mg/dL,2219
1,50363251,141168,1133,3,PT - INR,2.5,2.5,ratio,NaN,1208
2,49149139,141168,2026,1,magnesium,2.0,2.0,mg/dL,mg/dL,2090
3,50363250,141168,1133,3,PT,26.6,26.6,sec,sec,1208
4,66695374,141168,2141,7,pH,7.2,7.20,NaN,Units,2155


In [ ]:
# Pull out dataframe for labresults 'WBC x 1000', 'lactate', 'creatinine'


In [18]:
# converts all occurances of urinary creatinine in labname col to creatinine
# alternativy check labtypeid creatinine = 1,  urinary creatinine = 4

lab.loc[lab['labname'].str.contains("urinary creatinine"), 'labname'] = "creatinine"
lab.loc[(lab['labname'].str.contains("creatinine"))]

,patientunitstayid,labtypeid,labname,labresult,labmeasurenamesystem,labmeasurenameinterface
labresultoffset,,,,,,
1701,141168,4,creatinine,173.12,mg/dL,mg/dL
2026,141168,1,creatinine,2.95,mg/dL,mg/dL
1133,141168,1,creatinine,2.30,mg/dL,mg/dL
516,141168,1,creatinine,1.95,mg/dL,mg/dL
-280,141178,1,creatinine,0.70,mg/dL,mg/dL
...,...,...,...,...,...,...
10778,180812,1,creatinine,1.32,mg/dL,mg/dL
4837,180812,1,creatinine,1.25,mg/dL,mg/dL
13693,180812,1,creatinine,1.33,mg/dL,mg/dL


In [14]:
# showing all results for parient with id 154910

lab['patientunitstayid'] = lab['patientunitstayid'].astype(str)
lab.loc[(lab['patientunitstayid'].str.contains("154910"))]


The `intakeOutput` table provides Intake and output recorded for patients entered from the nursing flowsheet.

In [11]:
intakeOutput = pd.read_csv('../eICU' + database_type + '/intakeOutput.csv', nrows=999999)
columns = ['patientunitstayid', 'intakeoutputoffset', 'intaketotal', 'outputtotal', 'celllabel']
intakeOutput = intakeOutput[columns].set_index('intakeoutputoffset')
intakeOutput.sort_values(by='intakeoutputoffset')
intakeOutputUrine = intakeOutput.loc[(intakeOutput['celllabel'].str.contains("Urine"))]
intakeOutput.head()

,patientunitstayid,intaketotal,outputtotal,celllabel
intakeoutputoffset,,,,
661,205505,0.0,0.0,Bodyweight (lb)
661,205505,0.0,0.0,Bodyweight (kg)
3523,150708,727.0,0.0,P.O.
3523,150708,727.0,0.0,Volume (mL)-sodium chloride 0.9 % flush IVPB 5...
24801,158159,120.0,0.0,P.O.


In [13]:
intakeOutputUrine = pd.DataFrame.merge(patient, intakeOutputUrine,on='patientunitstayid')
intakeOutputUrine

,patientunitstayid,admissionweight_x,admissionweight_y,intaketotal,outputtotal,celllabel
0,141179,NaN,NaN,0.0,700.0,Urine
1,141179,NaN,NaN,0.0,500.0,Urine
2,141179,NaN,NaN,0.0,1000.0,Urine
3,141179,NaN,NaN,0.0,700.0,Urine
4,141194,73.90,73.90,290.4,100.0,Urine
...,...,...,...,...,...,...
207342,425009,65.80,65.80,0.0,300.0,Urine
207343,425010,65.80,65.80,0.0,150.0,Urine
207344,425072,93.90,93.90,0.0,350.0,Urine
207345,425072,93.90,93.90,0.0,0.0,Urine
